In [1]:
from sklearn.model_selection import train_test_split #分割训练集和测试集
from sklearn.neighbors import KNeighborsClassifier   #k-近邻算法
from sklearn.model_selection import GridSearchCV  #网格搜索
from sklearn.preprocessing import StandardScaler #特征缩放
import pandas as pd

In [10]:
def knncls():
    """
    k-近邻预测用户签到位置
    """
    #1.读取数据
    data = pd.read_csv("./train.csv")
    #2.处理数据
    
        #2.1 缩小数据范围（少量的数据容易操作）
    data = data.query("x > 1.0 & x < 1.25 & y >2.5 & y < 2.75")
        #2.2 处理时间数据
    time_value = pd.to_datetime(data['time'],unit='s')
        #2.3 把日期格式转换成字典格式
    time_value = pd.DatetimeIndex(time_value)
    #3.构造一些特征
    data['day'] = time_value.day
    data['hour'] = time_value.hour
    data['weekday'] = time_value.weekday
        #3.1 把时间戳特征删除
    data = data.drop(['time'],axis=1)
        #3.2 把签到数量少于n个目标位置删除
    place_count = data.groupby('place_id').count()
    tf = place_count[place_count.row_id > 3].reset_index()  #reset_index()把索引设置为列
    data = data[data['place_id'].isin(tf.place_id)]
        #3.3 删除无关特征
    data = data.drop(['row_id'],axis=1)
        #3.3 取出数据中的特征值和目标值
    y = data['place_id']
    x = data.drop(['place_id'],axis=1)
        #3.4 进行数据的分割训练集和测试集
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)
    
    #4.特征工程（标准化）
    std = StandardScaler()
        #4.1 对测试集和训练集的特征值进行标准化
    x_train = std.fit_transform(x_train)
    x_test = std.transform(x_test)
     
    
    #5. 进行算法流程（超参数）
    knn = KNeighborsClassifier()# n_neighbors 为 k值
    
#     #5.1 写入训练集数据 
#     knn.fit(x_train,y_train)
    
#     #5.2 得到预测结果
#     y_predict = knn.predict(x_test)
#     print("预测的目标签到位置为：",y_predict)

#     #得出准确率
#     print("预测的准确率：",knn.score(x_test,y_test))

    #5.1 构造一些参数的值进行索引
    param_grid = {'n_neighbors':[3,5,10]}
    #5.2 进行网格搜索
    gc = GridSearchCV(knn,param_grid=param_grid,cv=2)
    #5.3 写入训练集数据
    gc.fit(x_train,y_train)
    # 预测准确率
    print('在测试集上准确率:',gc.score(x_test,y_test))
    
    print('在交叉验证当中最好的结果：',gc.best_score_)
    print('选择最好的模型是：',gc.best_estimator_)
    print('每个超参数每次交叉验证的结果：',gc.cv_results_)
    return None
    
knncls()

/home/tang/视频/北京15期就业班/源码/venv/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/tang/视频/北京15期就业班/源码/venv/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/tang/视频/北京15期就业班/源码/venv/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

在测试集上准确率: 0.4777777777777778
在交叉验证当中最好的结果： 0.4489281210592686
选择最好的模型是： KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')
每个超参数每次交叉验证的结果： {'mean_fit_time': array([0.00551116, 0.00405693, 0.00395548]), 'std_fit_time': array([1.53243542e-03, 9.96589661e-05, 7.66515732e-05]), 'mean_score_time': array([0.22068048, 0.2312746 , 0.25882566]), 'std_score_time': array([0.00798512, 0.00310636, 0.00673759]), 'param_n_neighbors': masked_array(data=[3, 5, 10],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 10}], 'split0_test_score': array([0.4234192 , 0.43419204, 0.44215457]), 'split1_test_score': array([0.42511539, 0.45217253, 0.4558332 ]), 'mean_test_score': array([0.42425914, 0.44309584, 0.44892812]), 'std_test_score': array([0.00084805, 0.00898983, 0